<h1 align="center">Implementing Apriori from Scratch</h1>

#### Creators: Uzair Tahamid Siam

##### Reference:

Data Mining Concepts and Techniques by Jiawei Han, Micheline Kamber and Jian Pei

---

# NOTES:

The dataset is fairly *large* and the support counts are fairly *large* as well. As a result it takes really long to generate frequent itemsets if the min support is too low. 

Some min supports that I used to test and their runtimes are as follows:

| min support | runtime (s) |
| --- | ----------- |
| <1000 | too long |
| 1000 | 72 |
| 2000 | 30 |
| 3000 | 12 |
| 4000 | 9 |
| 5000 | 5 |


*** I ran the algorithm using the the pandas dataframe and it was a lot slower than reading from a text file so I decided to read and append to a list from a .txt file instead of using the dataframe

In [5]:
from collections import defaultdict
from itertools import combinations
from typing import Any

class Apriori:
    def __init__(self, pathToFile, minSupport):
        self.transactions = self._readFile(filename=pathToFile)
        self.minSupport = minSupport
        
    def _readFile(self, filename)->list:
        '''
        Function to return an array of sets read from a file
        
        Parameters
        ----------
        filename: str
            Path to .txt file 
            
        Returns
        ----------
        data_rows: list
            Returns list of transaction sets 
        '''
        file = open(filename, 'r')

        dataRows = []
        
        for line in file:
            line = line.strip().rstrip(',')
            record = set(line.split(', '))
            dataRows.append(record)
            
                
        return dataRows


    def _isSubList(self, patternList, subList)->bool:
        '''
        Function to check if a certain pattern is present in a sublist

        Parameters
        ----------
        patternList: list
            parent pattern list to  
            
        Returns
        ----------
        isSublist: bool
            Returns True if sublist present in pattern list else False 
        '''
        while subList:
            if subList[-1] in patternList:
                itemToRemove = subList.pop()
                patternList.remove(itemToRemove)
                if not subList:
                    return True
            else:
                return False

    def _calcItemFreq(self, transactions) -> defaultdict[Any, int]:
        '''
        Function to count the number of occurences of an item in the transaction dataset
        
        Parameters
        ----------
        transactions: set
            Sets of items in the dataset
            
        Returns
        ----------
        itemCount: dict
            Returns a dictionary of counts associated to each item
        '''
        itemCount = defaultdict(int)
        for transaction in transactions:
            for item in transaction:
                itemCount[item] = 1 if item not in itemCount.keys() else itemCount[item] + 1
        return itemCount


    def _getCombinations(self, num, dataset=None)->list[tuple]:
        '''
        Function to get combinations of items
        
        Parameters
        ----------
        num: int
            Length of combination. num = 2 returns a list of combinations of 2 items and
            num = 3 of 3 items
            
        dataset: iterable 
            Iterable to create combinations out of
            
        Returns
        ----------
            List of combinations
        '''
        def _isSubset(combinationList):
            for i in range(len(combinationList)):
                samp = list(combinationList[:])
                del samp[i]
                if tuple(samp) not in dataset:
                    return False
            return True

        if num == 2:
            return [i for i in combinations(dataset, num)]
        else:
            freq = self._genFrequentSets(self._calcItemFreq(
                [list(i) for i in dataset]), minSupport = num - 1)
            if len(freq) < num:
                return []
            else:
                return [i for i in combinations(freq.keys(), num) if _isSubset(i)]


    def _genFrequencyDict(self, transactionList, combinationList, combinationFrequencyDict=defaultdict(lambda: 0))->Any:
        '''
        Parameters
        ----------
        transactionList: iterable
        
        
        combinationList: iterable
        
        
        combinationFrequencyDict: dict
        
        
        Returns
        ----------
        combinationFrequencyDict: dict
            Key: Value pair with Key being a certain combination and Value being the support count of
            the combination
        
        '''
        for i in combinationList:
            for j in transactionList:
                if self._isSubList(list(j), list(i)):
                    combinationFrequencyDict[i] += 1

        return combinationFrequencyDict


    def _genFrequentSets(self, transaction, minSupport)->dict:
        '''
        Function to generate frequent set
        
        Parameters
        ----------
        transactions: set
            Sets of items in the dataset
            
        Returns
        ----------
        freqSet: dict
            Returns a dictionary of sets with support count higher than min support
        '''
        
        return {key: supCount for key, supCount in transaction.items() if supCount >= minSupport}


    def apriori(self, writeToFile=False)->list:
        '''
        Main apriori function to generate frequent sets and their support counts
        
        Parameters
        ----------
        transactions: set
        
        minSupport: int
        
        writeToFile: bool
        
        
        Returns
        ----------
        resList: List of key value pairs of {transaction: support_count}
        
        '''
        frequentItemSets = self._genFrequentSets(self._calcItemFreq(self.transactions), minSupport=self.minSupport)
        k = 2
        while True:
            combinationList = self._getCombinations(num=k, dataset=frequentItemSets)
            frequencyDictionary = self._genFrequencyDict(self.transactions, combinationList)
            frequentItemSets = self._genFrequentSets(frequencyDictionary, minSupport=self.minSupport)
            k += 1
            if not combinationList:
                break
            if not frequentItemSets:
                break

        fw = open("./data/apriori_results.txt", 'w', encoding='utf-8')
        resList = [] 
        for k, v in frequentItemSets.items():
            resList.append({k:v})
            if writeToFile:
                res = f"{k} | support: {v}"
                fw.writelines(res + '\n')
        return resList


    def printAssociationsWithLessThanEqualTo50K(self, assocs)->None:
        '''
        Prints out the associations and the support counts
        
        Parameters
        ----------
        assocs: list
            List of associations
            
        Returns
        ----------
        None
        '''
        for assoc in assocs:
            if "<=50K" in str(assoc.keys()):
                print(f"{assoc}")

    def printAssociationsWithMoreThan50K(self, assocs)->None:
        '''
        Prints out the associations and the support counts
        
        Parameters
        ----------
        assocs: list
            List of associations
            
        Returns
        ----------
        None
        '''
        for assoc in assocs:
            if ">50K" in str(assoc.keys()):
                print(f"{assoc}")



In [6]:
apriori = Apriori(pathToFile='data/processed_data.txt', minSupport=2000)
s = apriori.apriori(writeToFile=False)

In [8]:
apriori.printAssociationsWithMoreThan50K(s)

{('United-States', '>50K'): 6995}
{('Bachelors', '>50K'): 2126}
{('White', '>50K'): 6839}
{('adult', '>50K'): 2997}
{('middle-aged', '>50K'): 4058}
{('Husband', '>50K'): 5679}
{('Married-civ-spouse', '>50K'): 6399}
{('Male,0,0,over-time', '>50K'): 4155}
{('United-States', 'White', '>50K'): 6539}
{('United-States', 'adult', '>50K'): 2770}
{('United-States', 'middle-aged', '>50K'): 3805}
{('United-States', 'Husband', '>50K'): 5305}
{('United-States', 'Married-civ-spouse', '>50K'): 5959}
{('United-States', 'Male,0,0,over-time', '>50K'): 3874}
{('White', 'adult', '>50K'): 2726}
{('White', 'middle-aged', '>50K'): 3688}
{('White', 'Husband', '>50K'): 5226}
{('White', 'Married-civ-spouse', '>50K'): 5859}
{('White', 'Male,0,0,over-time', '>50K'): 3817}
{('adult', 'Husband', '>50K'): 2118}
{('adult', 'Married-civ-spouse', '>50K'): 2506}
{('middle-aged', 'Husband', '>50K'): 3199}
{('middle-aged', 'Married-civ-spouse', '>50K'): 3511}
{('middle-aged', 'Male,0,0,over-time', '>50K'): 2279}
{('Husban